In [34]:
# var inspector (install in anaconda prompt):
#   conda install -c conda-forge jupyterlab-variableinspector

In [35]:
# %conda install tensorflow
# %conda install tensorflow-gpu
# %conda install numpy
# %conda install pandas
# %conda install -c conda-forge matplotlib
# %conda install keras
# %conda install Pillow

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3598490856552459099,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5767102464
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9049777262286507189
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"]

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [4]:
# image size: 1080*1920(9:16)
INPUT_HEIGHT = 144
INPUT_WIDTH = 256
INPUT_CHANNEL = 3
INPUT_SHAPE = (INPUT_HEIGHT,INPUT_WIDTH,INPUT_CHANNEL)

OUTPUT_HEIGHT = INPUT_HEIGHT
OUTPUT_WIDTH = INPUT_WIDTH
OUTPUT_CHANNEL = INPUT_CHANNEL
OUTPUT_SHAPE = INPUT_SHAPE

In [5]:
import glob
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# FIXME: tf.data.Dataset maybe faster
# e.g. dir_name = 'data/stimuli.out/'
def x_from_dir(dir_name):
    x_paths = glob.glob(dir_name + '*.jpg', recursive=False)[:2000]
    idx = 0
    
    # initialize
    first_img = load_img(x_paths[0], target_size=(INPUT_HEIGHT, INPUT_WIDTH))
    x = img_to_array(first_img).reshape(1,INPUT_HEIGHT,INPUT_WIDTH,INPUT_CHANNEL)
    # remaining
    for path in x_paths[1:]:
        img = load_img(path, target_size=(INPUT_HEIGHT, INPUT_WIDTH))
        arr = img_to_array(img).reshape(1,INPUT_HEIGHT,INPUT_WIDTH, INPUT_CHANNEL)
        x = np.append(x, arr, axis=0)
        if idx%250 == 0:
            print(f'now loading {idx}-th image')
        idx = idx + 1

    return x / 255

# FIXME: tf.data.Dataset maybe faster
# e.g. dir_name = 'data/heatmap/'
def y_from_dir(dir_name):
    y_paths = glob.glob(dir_name + '*_heatmap.png', recursive=False)
    idx = 0
    
    # initialize
    first_img = load_img(y_paths[0], color_mode='grayscale', target_size=(OUTPUT_HEIGHT, OUTPUT_WIDTH))
    y = img_to_array(first_img).reshape(1,OUTPUT_HEIGHT,OUTPUT_WIDTH,1)
    # remaining
    for path in y_paths[1:]:
        img = load_img(path, color_mode='grayscale', target_size=(OUTPUT_HEIGHT, OUTPUT_WIDTH))
        arr = img_to_array(img).reshape(1,OUTPUT_HEIGHT,OUTPUT_WIDTH,1)
        y = np.append(y, arr, axis=0)
        if idx%100 == 0:
            print(f'now loading {idx}-th image')
        idx = idx + 1

    return y / 256

def salicon_y_from_dir(dir_name):
    y_paths = glob.glob(dir_name + '*.png', recursive=False)[:2000]
    idx = 0
    
    # initialize
    first_img = load_img(y_paths[0], color_mode='grayscale', target_size=(OUTPUT_HEIGHT, OUTPUT_WIDTH))
    y = img_to_array(first_img).reshape(1,OUTPUT_HEIGHT,OUTPUT_WIDTH,1)
    # remaining
    for path in y_paths[1:]:
        img = load_img(path, color_mode='grayscale', target_size=(OUTPUT_HEIGHT, OUTPUT_WIDTH))
        arr = img_to_array(img).reshape(1,OUTPUT_HEIGHT,OUTPUT_WIDTH,1)
        y = np.append(y, arr, axis=0)
        if idx%250 == 0:
            print(f'now loading {idx}-th image')
        idx = idx + 1

    return y / 256

In [49]:
# https://jpn.pioneer/ja/strengths/crdl/rd/pdf/2020-1.pdf
from tensorflow.keras import datasets, layers, models

model = models.Sequential()

model.add(layers.Conv2D(64, (7, 7), activation='relu', input_shape=INPUT_SHAPE, padding='same'))
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), padding='same'))
model.add(layers.Conv2D(128, (5, 5), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (5, 5), activation='relu', padding='same'))
# model.add(layers.Conv2D(128, (5, 5), activation='relu', padding='same')) #
# model.add(layers.MaxPooling2D((2, 2), padding='same'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D((2, 2), padding='same'))
# model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# 5*5 dilated convolution
model.add(layers.Conv2D(512, (5, 5), activation='relu', strides=1, padding='same', dilation_rate=1))
model.add(layers.Conv2D(512, (5, 5), activation='relu', strides=1, padding='same', dilation_rate=1))
model.add(layers.Conv2D(512, (5, 5), activation='relu', strides=1, padding='same', dilation_rate=1))
# model.add(layers.UpSampling2D((2, 2)))
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.UpSampling2D((2, 2)))
# model.add(layers.Conv2D(128, (5, 5), activation='relu', padding='same'))
# model.add(layers.Conv2D(128, (5, 5), activation='relu', padding='same'))
# model.add(layers.Conv2D(128, (5, 5), activation='relu', padding='same')) #
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (7, 7), activation='relu', padding='same'))
model.add(layers.Conv2D(1, (7, 7), activation='relu', padding='same'))

model.summary()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 144, 256, 64)      9472      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 144, 256, 64)      102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 72, 128, 64)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 72, 128, 128)      204928    
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 72, 128, 128)      409728    
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 72, 128, 512)      1638912   
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 128, 512)     

In [6]:
# https://jpn.pioneer/ja/strengths/crdl/rd/pdf/2020-1.pdf
from tensorflow.keras import datasets, layers, models

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=INPUT_SHAPE, padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(1, (3, 3), activation='relu', padding='same'))

model.summary()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 144, 256, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 144, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 72, 128, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 128, 128)      73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 128, 128)      147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 128, 128)      147584    
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 144, 256, 128)     0

In [15]:
# salicon dataset
salicon_x_dir = 'data/salicon_images/'
salicon_x = x_from_dir(salicon_x_dir)
salicon_y_dir = 'data/salicon_heatmap/'
salicon_y = salicon_y_from_dir(salicon_y_dir)

now loading 0-th image


KeyboardInterrupt: 

In [7]:
x_dir = 'data/stimuli.out/'
x = x_from_dir(x_dir)
y_dir = 'data/heatmap/'
y = y_from_dir(y_dir)

now loading 0-th image
now loading 250-th image
now loading 0-th image
now loading 100-th image
now loading 200-th image


## 1st split

In [8]:
# slice all data
TRAIN_SIZE = 240
x_train = x[:TRAIN_SIZE]
x_test = x[TRAIN_SIZE:]
y_train = y[:TRAIN_SIZE]
y_test = y[TRAIN_SIZE:]
model.fit(x_train, y_train, batch_size=10, epochs=10)

Epoch 1/10
24/24 [==============================] - 8s 89ms/step - loss: 0.1876 - accuracy: 0.6930
Epoch 2/10
24/24 [==============================] - 2s 98ms/step - loss: 0.1627 - accuracy: 0.6944
Epoch 3/10
24/24 [==============================] - 2s 91ms/step - loss: 0.1619 - accuracy: 0.6944
Epoch 4/10
24/24 [==============================] - 2s 92ms/step - loss: 0.1612 - accuracy: 0.6944
Epoch 5/10
24/24 [==============================] - 2s 92ms/step - loss: 0.1610 - accuracy: 0.6944
Epoch 6/10
24/24 [==============================] - 2s 91ms/step - loss: 0.1601 - accuracy: 0.6944
Epoch 7/10
24/24 [==============================] - 2s 88ms/step - loss: 0.1598 - accuracy: 0.6944
Epoch 8/10
24/24 [==============================] - 2s 90ms/step - loss: 0.1597 - accuracy: 0.6944
Epoch 9/10
24/24 [==============================] - 2s 90ms/step - loss: 0.1590 - accuracy: 0.6944
Epoch 10/10
24/24 [==============================] - 2s 89ms/step - loss: 0.1586 - accuracy: 0.6944


In [13]:
# slice all data
TRAIN_SIZE = 270
x_train = x[:TRAIN_SIZE]
x_test = x[TRAIN_SIZE:]
y_train = y[:TRAIN_SIZE]
y_test = y[TRAIN_SIZE:]
x_train_with_salicon = np.append(x_train, salicon_x[:2000], axis=0)
y_train_with_salicon = np.append(y_train, salicon_y[:2000], axis=0)
model.fit(x_train_with_salicon, y_train_with_salicon, batch_size=10, epochs=10)

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[10,64,108,192] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_1/max_pooling2d_3/MaxPool (defined at \AppData\Local\Temp\ipykernel_4816\3658087579.py:9) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_2767]

Function call stack:
train_function


In [9]:
from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))
    # img.save('data/result/' + str(i) + '.png')

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255

    # fig, ax = plt.subplots()
    # ax.imshow(heat_sq)
    # fig, ax = plt.subplots()
    # ax.imshow(true)
    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1
    
print(f"mean l2 distance: {l2 / len(y_pred)}")

mean l2 distance: 29.392565155029295


## 2nd split

In [9]:
# slice all data
x_train = np.concatenate([ x[:180], x[240:] ])
x_test = x[180:240]
y_train = np.concatenate([ y[:180], y[240:] ])
y_test = y[180:240]
model.fit(x_train, y_train, batch_size=10, epochs=10)

from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))
    # img.save('data/result/' + str(i) + '.png')

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255
    
    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1
    
print(f"mean l2 distance: {l2 / len(y_pred)}")

Epoch 1/10
24/24 [==============================] - 9s 111ms/step - loss: 0.2251 - accuracy: 0.6936
Epoch 2/10
24/24 [==============================] - 3s 111ms/step - loss: 0.1611 - accuracy: 0.6970
Epoch 3/10
24/24 [==============================] - 3s 110ms/step - loss: 0.1590 - accuracy: 0.6970
Epoch 4/10
24/24 [==============================] - 3s 112ms/step - loss: 0.1590 - accuracy: 0.6970
Epoch 5/10
24/24 [==============================] - 3s 111ms/step - loss: 0.1582 - accuracy: 0.6970
Epoch 6/10
24/24 [==============================] - 3s 114ms/step - loss: 0.1580 - accuracy: 0.6970
Epoch 7/10
24/24 [==============================] - 3s 110ms/step - loss: 0.1574 - accuracy: 0.6970
Epoch 8/10
24/24 [==============================] - 3s 109ms/step - loss: 0.1567 - accuracy: 0.6970
Epoch 9/10
24/24 [==============================] - 3s 112ms/step - loss: 0.1564 - accuracy: 0.6970
Epoch 10/10
24/24 [==============================] - 3s 108ms/step - loss: 0.1556 - accuracy: 0.6970

## 3rd split

In [8]:
# slice all data
x_train = np.concatenate([ x[:120], x[180:] ])
x_test = x[120:180]
y_train = np.concatenate([ y[:120], y[180:] ])
y_test = y[120:180]
model.fit(x_train, y_train, batch_size=10, epochs=10)

from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))
    # img.save('data/result/' + str(i) + '.png')

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255

    # fig, ax = plt.subplots()
    # ax.imshow(heat_sq)
    # fig, ax = plt.subplots()
    # ax.imshow(true)
    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1

Epoch 1/10
24/24 [==============================] - 10s 112ms/step - loss: 0.1748 - accuracy: 0.6967
Epoch 2/10
24/24 [==============================] - 3s 112ms/step - loss: 0.1619 - accuracy: 0.6986
Epoch 3/10
24/24 [==============================] - 3s 112ms/step - loss: 0.1609 - accuracy: 0.6986
Epoch 4/10
24/24 [==============================] - 3s 111ms/step - loss: 0.1606 - accuracy: 0.6986
Epoch 5/10
24/24 [==============================] - 3s 111ms/step - loss: 0.1598 - accuracy: 0.6986
Epoch 6/10
24/24 [==============================] - 3s 112ms/step - loss: 0.1591 - accuracy: 0.6986
Epoch 7/10
24/24 [==============================] - 3s 114ms/step - loss: 0.1590 - accuracy: 0.6986
Epoch 8/10
24/24 [==============================] - 3s 111ms/step - loss: 0.1579 - accuracy: 0.6986
Epoch 9/10
24/24 [==============================] - 3s 111ms/step - loss: 0.1574 - accuracy: 0.6986
Epoch 10/10
24/24 [==============================] - 3s 111ms/step - loss: 0.1580 - accuracy: 0.698

In [9]:
print(f"mean l2 distance: {l2 / len(y_pred)}")

mean l2 distance: 29.75395892461141


## 4th split

In [9]:
# slice all data
x_train = np.concatenate([ x[:60], x[120:] ])
x_test = x[60:120]
y_train = np.concatenate([ y[:60], y[120:] ])
y_test = y[60:120]
model.fit(x_train, y_train, batch_size=10, epochs=10)

from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))
    # img.save('data/result/' + str(i) + '.png')

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255

    # fig, ax = plt.subplots()
    # ax.imshow(heat_sq)
    # fig, ax = plt.subplots()
    # ax.imshow(true)
    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1

Epoch 1/10
24/24 [==============================] - 9s 103ms/step - loss: 0.1861 - accuracy: 0.6818
Epoch 2/10
24/24 [==============================] - 2s 102ms/step - loss: 0.1629 - accuracy: 0.6901
Epoch 3/10
24/24 [==============================] - 3s 106ms/step - loss: 0.1615 - accuracy: 0.6901
Epoch 4/10
24/24 [==============================] - 3s 105ms/step - loss: 0.1617 - accuracy: 0.6901
Epoch 5/10
24/24 [==============================] - 3s 105ms/step - loss: 0.1586 - accuracy: 0.6901
Epoch 6/10
24/24 [==============================] - 2s 103ms/step - loss: 0.1589 - accuracy: 0.6901
Epoch 7/10
24/24 [==============================] - 3s 104ms/step - loss: 0.1580 - accuracy: 0.6901
Epoch 8/10
24/24 [==============================] - 2s 103ms/step - loss: 0.1570 - accuracy: 0.6901
Epoch 9/10
24/24 [==============================] - 2s 102ms/step - loss: 0.1572 - accuracy: 0.6901
Epoch 10/10
24/24 [==============================] - 2s 102ms/step - loss: 0.1580 - accuracy: 0.6901

In [10]:
print(f"mean l2 distance: {l2 / len(y_pred)}")

mean l2 distance: 28.604297574361166


## 5th split

In [8]:
# slice all data
x_train = x[60:]
x_test = x[:60]
y_train = y[60:]
y_test = y[:60]
model.fit(x_train, y_train, batch_size=10, epochs=10)

from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255

    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1

Epoch 1/10
24/24 [==============================] - 9s 102ms/step - loss: 0.1933 - accuracy: 0.6890
Epoch 2/10
24/24 [==============================] - 2s 103ms/step - loss: 0.1627 - accuracy: 0.6899
Epoch 3/10
24/24 [==============================] - 2s 102ms/step - loss: 0.1607 - accuracy: 0.6899
Epoch 4/10
24/24 [==============================] - 2s 103ms/step - loss: 0.1591 - accuracy: 0.6899
Epoch 5/10
24/24 [==============================] - 3s 105ms/step - loss: 0.1594 - accuracy: 0.6899
Epoch 6/10
24/24 [==============================] - 2s 102ms/step - loss: 0.1595 - accuracy: 0.6899
Epoch 7/10
24/24 [==============================] - 3s 105ms/step - loss: 0.1609 - accuracy: 0.6899
Epoch 8/10
24/24 [==============================] - 2s 103ms/step - loss: 0.1588 - accuracy: 0.6899
Epoch 9/10
24/24 [==============================] - 2s 103ms/step - loss: 0.1578 - accuracy: 0.68990s - loss: 0.158
Epoch 10/10
24/24 [==============================] - 2s 102ms/step - loss: 0.1564 - 

In [9]:
print(f"mean l2 distance: {l2 / len(y_pred)}")

mean l2 distance: 27.67216796875
